## 목차
### 1. Library Import
### 2. 학습데이터 생성
### 3. Light-gbm 모델 훈련
### 4. 교차 검증 점수 확인
### 5. 제출 파일 생성

# 1. Library Import

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
from google.colab import drive
drive.mount('/content/gdriv')
PATH ='/content/gdriv/My Drive/LG/data/'

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)




Drive already mounted at /content/gdriv; to attempt to forcibly remount, call drive.mount("/content/gdriv", force_remount=True).


# 2. 학습 데이터 생성

## 2.1 train_err

* train_err_data.csv 파일의 errtype column의 value_counts를 카운트하여 학습에 사용.  
* 주어진 기간동안 user_id별로 peroblem가 한 번이라도 발생했는 지를 확인.  

In [2]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [3]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

user_id            time model_nm       fwver  errtype errcode
0    10000  20201101025616  model_3  05.15.2138       15       1
1    10000  20201101030309  model_3  05.15.2138       12       1
2    10000  20201101030309  model_3  05.15.2138       11       1
3    10000  20201101050514  model_3  05.15.2138       16       1
4    10000  20201101050515  model_3  05.15.2138        4       0

In [4]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [5]:
# errtype_pick = [16, 26, 31, 40, 15, 5, 23, 14, 22, 34, 4, 12, 17, 42, 11, 33]

In [6]:
# train_err['errtype_pick'] = train_err['errtype'].isin(errtype_pick).astype('int64')

In [7]:
# train_err = train_err[train_err.errtype_pick == 1]
# train_err

In [8]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████| 16554663/16554663 [00:50<00:00, 329718.67it/s]


(15000, 42)

In [9]:
train_err.dropna(axis = 0, inplace=True)
train_err.shape

(16554662, 6)

In [10]:
items = train_err.errcode.unique().astype(str)
items

array(['1', '0', '2', ..., '4526', '3965', '25999'], dtype='<U39')

In [11]:
len(train_err.errcode.unique())

2805

In [12]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(train_err.errcode)
labels.shape

(16554662,)

In [13]:
train_err['errcode_label'] = labels

In [14]:
errcode_pick = [2, 4, 279, 370, 2728, 196,2566, 2796, 2755, 2537,2590, 2629,2647,2757,2609,2784,2475,2797,2734,     
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800, 2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153,2676]

In [15]:
train_err['errcode_pick'] = train_err['errcode_label'].isin(errcode_pick).astype('int64')

In [16]:
train_err.errcode_pick.value_counts()

1    16511265
0       43397
Name: errcode_pick, dtype: int64

In [17]:
train_err = train_err[train_err.errcode_pick == 1]
train_err

user_id            time model_nm  ... errcode  errcode_label errcode_pick
0           10000  20201101025616  model_3  ...       1              4            1
1           10000  20201101030309  model_3  ...       1              4            1
2           10000  20201101030309  model_3  ...       1              4            1
3           10000  20201101050514  model_3  ...       1              4            1
4           10000  20201101050515  model_3  ...       0              2            1
...           ...             ...      ...  ...     ...            ...          ...
16554658    24999  20201130163051  model_3  ...       1              4            1
16554659    24999  20201130172625  model_3  ...       1              4            1
16554660    24999  20201130172625  model_3  ...       0              2            1
16554661    24999  20201130172631  model_3  ...       0              2            1
16554662    24999  20201130210625  model_3  ...       1              4            1

[16511265 rows x 8 columns]

In [18]:
train_err.errcode_label.unique()

array([   4,    2,  279, 2755, 2728,  196, 1209,  176,  370, 2517, 2803,
       2796, 2768, 2799, 2771, 2804, 2798, 2518, 2500, 2537, 2598, 2579,
       2800, 2475, 2492, 2785, 2590, 2639, 2629, 2791, 2797, 2774, 2566,
       2676, 2647, 2609, 2776, 2153, 1537, 2729,  297, 2744, 2802, 2758,
       2763, 2784, 2757, 2734])

In [19]:
from sklearn.preprocessing import LabelEncoder

encoder2 = LabelEncoder()
encoder2.fit(train_err.errcode_label.unique())
labels = encoder2.transform(train_err.errcode_label)
labels.shape

(16511265,)

In [20]:
train_err['errcode_label_2'] = labels

In [21]:
train_err.errcode_label_2.value_counts().sort_index()

0     2599123
1     8906967
2       12040
3      263577
4      166382
5         982
6       91223
7       20249
8        7335
9        7392
10        629
11       5028
12     332422
13       9317
14     334018
15       4339
16       2087
17     129876
18     127744
19       4023
20       1279
21      64799
22      65992
23       1001
24      23452
25     897863
26        172
27         68
28        163
29     104138
30         35
31        105
32        157
33      11959
34      34631
35        138
36       1583
37         62
38       3914
39       1740
40     219238
41        187
42     100940
43    1835600
44       4011
45        392
46     110370
47       2523
Name: errcode_label_2, dtype: int64

In [22]:
len(train_err.errcode_label_2.unique())

48

In [23]:
id_code = train_err[['user_id','errcode_label_2']].values
code = np.zeros((train_user_number,48))

for person_idx, co in tqdm(id_code):
    # person_idx - train_user_id_min 위치에 person_idx, errcode에 해당하는 error값을 +1
    code[person_idx - train_user_id_min,co] += 1
code.shape

100%|██████████| 16511265/16511265 [00:46<00:00, 358893.20it/s]


(15000, 48)

In [24]:
type_code = np.append(error, code, axis=1)
type_code.shape

(15000, 90)

In [25]:
#fwver encoder
encoder_fw = LabelEncoder()
encoder_fw.fit(train_err.fwver.unique())
labels = encoder_fw.transform(train_err.fwver)
labels.shape

(16511265,)

In [26]:
train_err['fwver_label'] = labels

In [27]:
train_err.fwver_label.value_counts().sort_index()

0          29
1        2114
2     1505615
3         296
4         101
5         140
6     5203296
7         962
8      143655
9        2522
10         39
11          4
12       5423
13    2872302
14    1293128
15        909
16        408
17       3236
18         58
19     962913
20    2502266
21      33535
22       2509
23      23404
24         78
25       8592
26         12
27         18
28          9
29        705
30         23
31    1905869
32        191
33      31831
34       3334
35       1528
36        211
Name: fwver_label, dtype: int64

In [28]:
len(train_err.fwver_label.unique())

37

In [29]:
fwver_pick = [20,6,14,31,13,19,2,8,17,33,35,12,21,25,23]
train_err['fwver_pick'] = train_err['fwver_label'].isin(fwver_pick).astype('int64')
train_err = train_err[train_err.fwver_pick == 1]
train_err

user_id            time  ... fwver_label fwver_pick
0           10000  20201101025616  ...          31          1
1           10000  20201101030309  ...          31          1
2           10000  20201101030309  ...          31          1
3           10000  20201101050514  ...          31          1
4           10000  20201101050515  ...          31          1
...           ...             ...  ...         ...        ...
16554658    24999  20201130163051  ...          31          1
16554659    24999  20201130172625  ...          31          1
16554660    24999  20201130172625  ...          31          1
16554661    24999  20201130172631  ...          31          1
16554662    24999  20201130210625  ...          31          1

[16496593 rows x 11 columns]

In [30]:
#fwver encoder
encoder_fw2 = LabelEncoder()
encoder_fw2.fit(train_err.fwver_label.unique())
labels = encoder_fw2.transform(train_err.fwver_label)
labels.shape

(16496593,)

In [31]:
train_err['fwver_label_2'] = labels

In [32]:
id_fwver = train_err[['user_id','fwver_label_2']].values
fwver = np.zeros((train_user_number,15))

for person_idx, fw in tqdm(id_fwver):
    # person_idx - train_user_id_min 위치에 person_idx, errcode에 해당하는 error값을 +1
    fwver[person_idx - train_user_id_min,fw] += 1
fwver.shape

100%|██████████| 16496593/16496593 [00:44<00:00, 372587.02it/s]


(15000, 15)

In [33]:
type_code_fwver = np.append(type_code, fwver, axis=1)
type_code_fwver.shape

(15000, 105)

In [34]:
#참고) 일별 합산 데이터 생성 코드      
'''
train_err             = pd.read_csv(PATH+'train_err_data.csv')
train_err['datetime'] = train_err['time'].apply(make_datetime)

min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

id_error = train_err[['user_id','errtype','days']].values
error = np.zeros((train_user_number,42, 33))
for idx, err, days in tqdm(id_error):
    error[idx - train_user_id_min,err - 1, days] += 1
error.shape
'''

"\ntrain_err             = pd.read_csv(PATH+'train_err_data.csv')\ntrain_err['datetime'] = train_err['time'].apply(make_datetime)\n\nmin_day = train_err.datetime.min().date()\n\ntrain_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day\ntrain_err['days'] = train_err['days'].dt.days\ndisplay(train_err)\n\nid_error = train_err[['user_id','errtype','days']].values\nerror = np.zeros((train_user_number,42, 33))\nfor idx, err, days in tqdm(id_error):\n    error[idx - train_user_id_min,err - 1, days] += 1\nerror.shape\n"

## 2.2 problem

In [35]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

# 3. Light-gbm 모델 훈련

In [36]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = type_code_fwver
train_y = problem
del type_code_fwver, problem, code, error
print(train_x.shape)
print(train_y.shape)

(15000, 105)
(15000,)


In [37]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 2000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.813478	valid_0's pr_auc: 0.807381
[40]	valid_0's auc: 0.815552	valid_0's pr_auc: 0.811433
[60]	valid_0's auc: 0.81577	valid_0's pr_auc: 0.812607
[80]	valid_0's auc: 0.816207	valid_0's pr_auc: 0.81396
[100]	valid_0's auc: 0.815078	valid_0's pr_auc: 0.813762
[120]	valid_0's auc: 0.813358	valid_0's pr_auc: 0.812579
[140]	valid_0's auc: 0.81365	valid_0's pr_auc: 0.813415
Early stopping, best iteration is:
[53]	valid_0's auc: 0.816567	valid_0's pr_auc: 0.8126
Training until validation scores don't improve for 100 rounds.
[20]	valid_0's auc: 0.818759	valid_0's pr_auc: 0.376488
[40]	valid_0's auc: 0.821488	valid_0's pr_auc: 0.389804
[60]	valid_0's auc: 0.818752	valid_0's pr_auc: 0.399201
[80]	valid_0's auc: 0.822639	valid_0's pr_auc: 0.408643
[100]	valid_0's auc: 0.817196	valid_0's pr_auc: 0.40459
[120]	valid_0's auc: 0.817391	valid_0's pr_auc: 0.398549
[140]	valid_0's auc: 0.815861	valid_0's pr_auc: 0.393879

# 4. 교차검증 점수 확인

In [38]:
print(np.mean(auc_scores))

0.8182515524436285


# 5. 제출 파일 생성

In [39]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

user_id            time model_nm       fwver  errtype errcode
0    30000  20201101030227  model_1  04.16.3553       31       1
1    30000  20201101030227  model_1  04.16.3553       33       2
2    30000  20201101030228  model_1  04.16.3553       15       1
3    30000  20201101030256  model_1  04.16.3553       22       1
4    30000  20201101030300  model_1  04.16.3553       11       1

In [40]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [41]:
test_err.dropna(axis = 0, inplace=True)
test_err.shape

(16532644, 6)

In [42]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - test_user_id_min,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
print(test_x.shape)

100%|██████████| 16532644/16532644 [00:49<00:00, 332362.32it/s]

(14999, 42)


In [43]:
test_err

user_id            time model_nm       fwver  errtype errcode
0           30000  20201101030227  model_1  04.16.3553       31       1
1           30000  20201101030227  model_1  04.16.3553       33       2
2           30000  20201101030228  model_1  04.16.3553       15       1
3           30000  20201101030256  model_1  04.16.3553       22       1
4           30000  20201101030300  model_1  04.16.3553       11       1
...           ...             ...      ...         ...      ...     ...
16532643    44998  20201130210050  model_1  04.16.3553       40       0
16532644    44998  20201130211831  model_1  04.16.3553       31       1
16532645    44998  20201130211832  model_1  04.16.3553       15       1
16532646    44998  20201130212259  model_1  04.16.3553       16       1
16532647    44998  20201130212316  model_1  04.16.3553       31       0

[16532644 rows x 6 columns]

In [44]:
a = set(test_err.errcode).intersection(train_err.errcode)
print(a)

{'H-51046', 'standby', 'NFANDROID2', 'V-21008', 'connection timeout', '3', '95', 'active', '21', '80', '2', '14', '87', '4', 'S-64002', 'C-11017', '83', '84', '6', 'Q-64002', 'scanning timeout', '78', 'PHONE_ERR', '77', '5', 'terminate by peer user', '8.0', '13', 'U-82026', 'P-41007 ', '91', 'UNKNOWN', 'S-65002', '90', '89', 'connection fail for LMP response timout', '1', 'connectionterminated by local host', '85', '86', 'P-41007', '81', 'C-13053', 'connection fail to establish', 'B-A8002', 'S-61001', '79', '0'}


In [45]:
a = list(a)
print(a)

['H-51046', 'standby', 'NFANDROID2', 'V-21008', 'connection timeout', '3', '95', 'active', '21', '80', '2', '14', '87', '4', 'S-64002', 'C-11017', '83', '84', '6', 'Q-64002', 'scanning timeout', '78', 'PHONE_ERR', '77', '5', 'terminate by peer user', '8.0', '13', 'U-82026', 'P-41007 ', '91', 'UNKNOWN', 'S-65002', '90', '89', 'connection fail for LMP response timout', '1', 'connectionterminated by local host', '85', '86', 'P-41007', '81', 'C-13053', 'connection fail to establish', 'B-A8002', 'S-61001', '79', '0']


In [46]:
test_err.errcode.unique()

array(['1', '2', 'connection timeout', ..., '5803', '5444', '5691'],
      dtype=object)

In [47]:
test_err['errcode_pick'] = test_err['errcode'].isin(a).astype('int64')

In [48]:
test_err.errcode_pick.value_counts()

1    16497790
0       34854
Name: errcode_pick, dtype: int64

In [49]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errtype  errcode errcode_pick
0           30000  20201101030227  model_1  ...      31        1            1
1           30000  20201101030227  model_1  ...      33        2            1
2           30000  20201101030228  model_1  ...      15        1            1
3           30000  20201101030256  model_1  ...      22        1            1
4           30000  20201101030300  model_1  ...      11        1            1
...           ...             ...      ...  ...     ...      ...          ...
16532643    44998  20201130210050  model_1  ...      40        0            1
16532644    44998  20201130211831  model_1  ...      31        1            1
16532645    44998  20201130211832  model_1  ...      15        1            1
16532646    44998  20201130212259  model_1  ...      16        1            1
16532647    44998  20201130212316  model_1  ...      31        0            1

[16497790 rows x 7 columns]

In [50]:
labels = encoder.transform(test_err.errcode)
labels.shape

(16497790,)

In [51]:
test_err['errcode_label'] = labels

In [52]:
errcode_pick = [2, 4, 279, 370, 2728, 196,2566, 2796, 2755, 2537,2590, 2629,2647,2757,2609,2784,2475,2797,2734,     
                2799, 2803, 2771, 2517, 1209, 2500, 176, 
                2798, 2518, 2804, 2768, 2791, 297, 2802, 
                2492, 2776, 2785, 2800, 2598, 2579, 2744, 
                2763, 2639, 2774, 2758, 2729, 1537, 2153,2676]

In [53]:
test_err['errcode_pick'] = test_err['errcode_label'].isin(errcode_pick).astype('int64')

In [54]:
test_err = test_err[test_err.errcode_pick == 1]
test_err

user_id            time model_nm  ... errcode  errcode_pick errcode_label
0           30000  20201101030227  model_1  ...       1             1             4
1           30000  20201101030227  model_1  ...       2             1           279
2           30000  20201101030228  model_1  ...       1             1             4
3           30000  20201101030256  model_1  ...       1             1             4
4           30000  20201101030300  model_1  ...       1             1             4
...           ...             ...      ...  ...     ...           ...           ...
16532643    44998  20201130210050  model_1  ...       0             1             2
16532644    44998  20201130211831  model_1  ...       1             1             4
16532645    44998  20201130211832  model_1  ...       1             1             4
16532646    44998  20201130212259  model_1  ...       1             1             4
16532647    44998  20201130212316  model_1  ...       0             1             2

[16497790 rows x 8 columns]

In [55]:
test_err.errcode_label.value_counts().sort_index()

2       2565532
4       8750154
176       11740
196      316110
279      155401
297         997
370       86451
1209      19084
1537      12991
2153      13030
2475        561
2492       5059
2500     324186
2517       9427
2518     326179
2537       4252
2566       1836
2579     140642
2590     139255
2598       3164
2609       1016
2629      57718
2639      58820
2647        798
2676      17806
2728     855747
2729        211
2734         32
2744        113
2755     108244
2757         33
2758        142
2763        181
2768      11808
2771      35026
2774        136
2776       1636
2784         99
2785       3571
2791       1064
2796     225519
2797       1045
2798     109690
2799    1890632
2800      84809
2802        347
2803     139947
2804       5549
Name: errcode_label, dtype: int64

In [56]:
labels = encoder2.transform(test_err.errcode_label)
labels.shape

(16497790,)

In [57]:
test_err['errcode_label_2'] = labels

In [58]:
test_err.errcode_label_2.value_counts().sort_index()

0     2565532
1     8750154
2       11740
3      316110
4      155401
5         997
6       86451
7       19084
8       12991
9       13030
10        561
11       5059
12     324186
13       9427
14     326179
15       4252
16       1836
17     140642
18     139255
19       3164
20       1016
21      57718
22      58820
23        798
24      17806
25     855747
26        211
27         32
28        113
29     108244
30         33
31        142
32        181
33      11808
34      35026
35        136
36       1636
37         99
38       3571
39       1064
40     225519
41       1045
42     109690
43    1890632
44      84809
45        347
46     139947
47       5549
Name: errcode_label_2, dtype: int64

In [59]:
id_code = test_err[['user_id','errcode_label_2']].values
test_code = np.zeros((test_user_number,48))

for person_idx, code in tqdm(id_code):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_code[person_idx - test_user_id_min, code] += 1
test_code = test_code.reshape(test_code.shape[0],-1)
print(test_code.shape)

100%|██████████| 16497790/16497790 [00:45<00:00, 362082.94it/s]

(14999, 48)


In [60]:
test_type_code = np.append(test_x, test_code, axis=1)
test_type_code.shape

(14999, 90)

In [61]:
a = set(test_err.fwver).intersection(train_err.fwver)
print(a)

{'04.22.1750', '04.82.1684', '04.16.3571', '03.11.1167', '05.15.2138', '10', '04.33.1185', '04.22.1778', '04.33.1149', '04.22.1684', '04.73.2237', '04.16.3553', '05.66.3237', '04.82.1778', '04.33.1261'}


In [62]:
a = list(a)
print(a)

['04.22.1750', '04.82.1684', '04.16.3571', '03.11.1167', '05.15.2138', '10', '04.33.1185', '04.22.1778', '04.33.1149', '04.22.1684', '04.73.2237', '04.16.3553', '05.66.3237', '04.82.1778', '04.33.1261']


In [63]:
test_err['fwver_pick'] = test_err['fwver'].isin(a).astype('int64')

In [64]:
test_err = test_err[test_err.fwver_pick == 1]
test_err

user_id            time  ... errcode_label_2 fwver_pick
0           30000  20201101030227  ...               1          1
1           30000  20201101030227  ...               4          1
2           30000  20201101030228  ...               1          1
3           30000  20201101030256  ...               1          1
4           30000  20201101030300  ...               1          1
...           ...             ...  ...             ...        ...
16532643    44998  20201130210050  ...               0          1
16532644    44998  20201130211831  ...               1          1
16532645    44998  20201130211832  ...               1          1
16532646    44998  20201130212259  ...               1          1
16532647    44998  20201130212316  ...               0          1

[16474834 rows x 10 columns]

In [65]:
labels = encoder_fw.transform(test_err.fwver)
labels.shape

(16474834,)

In [66]:
test_err['fwver_label'] = labels

In [67]:
test_err.fwver_label.value_counts().sort_index()

2     1209624
6     5301382
8      145627
12       4950
13    2967105
14    1367304
17       6018
19     986904
20    2604917
21      20410
23      23002
25      12004
31    1799342
33      25684
35        561
Name: fwver_label, dtype: int64

In [68]:
labels = encoder_fw2.transform(test_err.fwver_label)
labels.shape

test_err['fwver_label_2'] = labels

In [69]:
test_err['fwver_label_2'].unique()

array([ 1,  2,  8, 12,  4,  5,  6,  0,  7,  3, 10, 11, 14, 13,  9])

In [70]:
id_fwver = test_err[['user_id','fwver_label_2']].values
test_fwver = np.zeros((test_user_number,15))

for person_idx, fwv in tqdm(id_fwver):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_fwver[person_idx - test_user_id_min, fwv] += 1
test_fwver = test_fwver.reshape(test_fwver.shape[0],-1)
print(test_fwver.shape)

100%|██████████| 16474834/16474834 [00:45<00:00, 358309.75it/s]

(14999, 15)


In [71]:
test_type_code_fw = np.append(test_type_code, test_fwver, axis=1)
test_type_code_fw.shape

(14999, 105)

In [72]:
# 예측
pred_y_list = []
for model in models:
    pred_y = model.predict(test_type_code_fw)
    pred_y_list.append(pred_y.reshape(-1,1))
    
pred_ensemble = np.mean(pred_y_list, axis = 0)

In [73]:
pred_ensemble

array([[0.93262794],
       [0.22003021],
       [0.55593083],
       ...,
       [0.78187945],
       [0.85290376],
       [0.41344504]])

In [74]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [75]:
sample_submssion['problem'] = pred_ensemble.reshape(-1)

In [76]:
sample_submssion.to_csv(PATH+"dacon_baseline_errcode_type_code_pick_fwver15_additive48.csv", index = False)
sample_submssion

user_id   problem
0        30000  0.932628
1        30001  0.220030
2        30002  0.555931
3        30003  0.769922
4        30004  0.871440
...        ...       ...
14994    44994  0.153722
14995    44995  0.331589
14996    44996  0.781879
14997    44997  0.852904
14998    44998  0.413445

[14999 rows x 2 columns]